In [3]:
# Energy Production and Usage
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [6]:
# Crude Oil by State Group/Area
s_ids = ['MCRFPCO2', 'MCRFPTX2', 'MCRFPND2', 'MCRFPNM2', 
         'MCRFPUS2', 'MCRFP3FM2']

d = {}
for series in s_ids:
    s = f'PET.{series}.M'
    url = f'http://api.eia.gov/series/?api_key={eia_key}&series_id={s}&start=198801'    
    r = requests.get(url).json()
    data = {pd.to_datetime(f'{i[0]}01'): i[1] for i in r['series'][0]['data']}
    d[r['series'][0]['series_id']] = data
    
df = pd.DataFrame(d).sort_index()
df['ND_NM_CO'] = df[['PET.MCRFPND2.M', 'PET.MCRFPNM2.M', 'PET.MCRFPCO2.M']].sum(axis=1)
df['USA'] = df['PET.MCRFPUS2.M']
df['TX'] = df['PET.MCRFPTX2.M']
df['GM'] = df['PET.MCRFP3FM2.M']
df['OTH'] = df['USA'] - df['TX'] - df['ND_NM_CO'] - df['GM']

data = df.loc['1989':,['ND_NM_CO', 'TX', 'OTH', 'GM', 'USA']].divide(1000, axis=1)
data.to_csv(data_dir / 'oil_prod.csv', index_label='date')
ltdate = dtxt(data.index[-1])['mon1']
ltval = data['USA'].iloc[-1]
text = (f'During {ltdate}, the US produced {ltval:.1f} million barrels per day.')
write_txt(text_dir / 'oil_prod.txt', text)
print(text)

During November 2021, the US produced 11.8 million barrels per day.


In [7]:
# Electricity Production by Major Source
s_ids = ['COW', 'NUC', 'TSN', 'HYC', 'HPS', 'NG', 'WND', 'GEO', 'WWW',
         'WAS', 'PEL', 'PC', 'ALL', 'AOR', 'OTH', 'SUN', 'OOG']

d = {}
for series in s_ids:
    s = f'ELEC.GEN.{series}-US-99.M'
    url = f'http://api.eia.gov/series/?api_key={eia_key}&series_id={s}&out=json'    
    r = requests.get(url).json()
    data = {pd.to_datetime(f'{i[0]}01'): i[1] for i in r['series'][0]['data']}
    d[r['series'][0]['series_id']] = data
    
df = pd.DataFrame(d).sort_index()

renew = ['SUN', 'HYC', 'WND', 'GEO', 'WAS', 'WWW']
renewables = [f'ELEC.GEN.{i}-US-99.M' for i in renew]
df['Renewables'] = df[renewables].sum(axis=1)

poth = ['PEL', 'PC', 'OTH', 'OOG', 'HPS']
petoth = [f'ELEC.GEN.{i}-US-99.M' for i in poth]
df['Petroleum and other'] = df[petoth].sum(axis=1)

bio = ['WWW', 'WAS']
biomass = [f'ELEC.GEN.{i}-US-99.M' for i in bio]
df['Biomass'] = df[biomass].sum(axis=1)

rename = {'ELEC.GEN.COW-US-99.M': 'Coal', 
          'ELEC.GEN.NUC-US-99.M': 'Nuclear', 
          'ELEC.GEN.NG-US-99.M': 'Natural Gas',
          'ELEC.GEN.HYC-US-99.M': 'Hydroelectric',
          'ELEC.GEN.SUN-US-99.M': 'Solar',
          'ELEC.GEN.WND-US-99.M': 'Wind',
          'ELEC.GEN.GEO-US-99.M': 'Geothermal'}
df.rename(rename, axis=1, inplace=True)

var = ['Coal', 'Natural Gas', 'Nuclear', 'Renewables', 'Petroleum and other',
       'Hydroelectric', 'Solar', 'Wind', 'Geothermal', 'Biomass']
data = df[var].rolling(12).sum().dropna().divide(1000, axis=1)
data.to_csv(data_dir / 'elec_prod.csv', index_label='date')

popgdp = pd.read_csv(data_dir / 'gdpjobslvl.csv', index_col='date', parse_dates=True)
pop = popgdp['pop'].dropna()
popch = (pop.iloc[-1] / pop.loc['2011'].mean() - 1) * 100
popinc = inc_dec_percent(popch)
gdp = popgdp['gdp'].dropna()
gdpch = (gdp.iloc[-1] / gdp.loc['2011'].mean() - 1) * 100
gdpinc = inc_dec_percent(gdpch)
major = ['Coal', 'Natural Gas', 'Nuclear', 'Renewables', 'Petroleum and other']
elec = data[major].sum(axis=1)
df2 = pd.DataFrame()
df2['elec'] = elec
df2['gdp'] = gdp
df2['pop'] = pop
df2['elecgdp'] = elec / gdp
df2['elecpop'] = elec / pop
df2 = ((df2 / df2.loc['2011-01-01']) - 1) * 100
df2.loc['2011':].to_csv(data_dir / 'elec.csv', index_label='date')

colors = {'gdp': 'green!60!blue', 
          'pop': 'blue!90!black', 
          'elec': 'red', 
          'elecpop': 'orange', 
          'elecgdp': 'cyan'}

adj = node_adjust(df2, colors)

#write_txt(text_dir / 'elec_nodes.txt', '')
node_file = open(text_dir / 'elec_nodes.txt', 'w')
for series, color in colors.items():
    if series in adj.keys():
        offset = adj[series]
    else:
        offset = 0
    node_file.write(end_node(df2[series].dropna(), color, percent=True))
node_file.close()

ltdate = dtxt(df.index[-1])['mon1']
ltval = data[major].iloc[-1].sum()
text = (f'Over the same period, the US population has {popinc} and real GDP has {gdpinc}. '+
        f'During the 12 months ending {ltdate}, the US generated {ltval:,.0f} billion kilowatt '+
        'hours of electricity.') 
write_txt(text_dir / 'elec_prod.txt', text)
print(text)

ngval = data['Natural Gas'].iloc[-1]
coalval = data['Coal'].iloc[-1]
nucval = data['Nuclear'].iloc[-1]
renval = data['Renewables'].iloc[-1]
text = (f'Over the year ending {ltdate}, {ngval:,.0f} billion kilowatt hours '+
        'of electricity were generated using natural gas (see\cbox{blue!40!cyan!60!white}), '+
        f'{coalval:,.0f} billion kilowatt hours were generated from coal '+
        f'(see\cbox{{brown}}), {nucval:,.0f} billion from nuclear (see\cbox{{red!80!blue!70!white}}), '+
        f'and {renval:,.0f} billion from renewable sources (see\cbox{{green!75!blue}}).')
write_txt(text_dir / 'elec_prod2.txt', text)
print(text)

hydval = data['Hydroelectric'].iloc[-1]
bioval = data['Biomass'].iloc[-1]
geoval = data['Geothermal'].iloc[-1]
sunval = data['Solar'].iloc[-1]
wndval = data['Wind'].iloc[-1]
text = (f'Among renewable energy sources, over the year ending {ltdate}, {hydval:,.0f} billion '+
        'kilowatt hours of electricity were generated with conventional hydroelectric (see\cbox{cyan}), '+
        f'{bioval:,.0f} billion kilowatt hours were generated from biomass (see\cbox{{brown!92!black}}), '+
        f'{geoval:,.0f} billion were generated from geothermal (see\cbox{{orange}}), '+
        f'{wndval:,.0f} billion from wind (see\cbox{{green!65!blue}}), '+
        f'and {sunval:,.0f} billion from solar (see\cbox{{yellow!70!orange}}).')
write_txt(text_dir / 'elec_prod3.txt', text)
print(text)

Over the same period, the US population has increased by 6.3 percent and real GDP has increased by 24.6 percent. During the 12 months ending November 2021, the US generated 4,123 billion kilowatt hours of electricity.
Over the year ending November 2021, 1,577 billion kilowatt hours of electricity were generated using natural gas (see\cbox{blue!40!cyan!60!white}), 918 billion kilowatt hours were generated from coal (see\cbox{brown}), 779 billion from nuclear (see\cbox{red!80!blue!70!white}), and 812 billion from renewable sources (see\cbox{green!75!blue}).
Among renewable energy sources, over the year ending November 2021, 256 billion kilowatt hours of electricity were generated with conventional hydroelectric (see\cbox{cyan}), 56 billion kilowatt hours were generated from biomass (see\cbox{brown!92!black}), 16 billion were generated from geothermal (see\cbox{orange}), 371 billion from wind (see\cbox{green!65!blue}), and 113 billion from solar (see\cbox{yellow!70!orange}).


In [8]:
# Electricity Retail Sales by Sector
s_ids = ['RES', 'COM', 'IND']

d = {}
for series in s_ids:
    s = f'ELEC.SALES.US-{series}.M'
    url = f'http://api.eia.gov/series/?api_key={eia_key}&series_id={s}&out=json'    
    r = requests.get(url).json()
    data = {pd.to_datetime(f'{i[0]}01'): i[1] for i in r['series'][0]['data']}
    d[r['series'][0]['series_id']] = data
    
df = pd.DataFrame(d).sort_index() / 1_000

rename = {'ELEC.SALES.US-RES.M': 'Residential', 
          'ELEC.SALES.US-COM.M': 'Commercial',
          'ELEC.SALES.US-IND.M': 'Industrial',}
df.rename(rename, axis=1, inplace=True)

data = df.rolling(12).sum().dropna()
data.to_csv(data_dir / 'elec_sales.csv', index_label='date')

write_txt(text_dir / 'elec_comm_node.txt', 
          end_node(data['Commercial'], 'blue!80!cyan', digits='comma'))
write_txt(text_dir / 'elec_ind_node.txt', 
          end_node(data['Industrial'], 'black!35!white', digits='comma'))
write_txt(text_dir / 'elec_res_node.txt', 
          end_node(data['Residential'], 'green!90!black', digits='comma'))